In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchsummary
import torchvision.models as models
import torch.quantization as quant

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


In [4]:
resnet18_static_model = torch.load('/home/tricus/task1/deeplearning-pytorch/trained_resnet18_model.pth', map_location=torch.device('cpu'))

In [5]:
quantized_model = torch.nn.Sequential(torch.quantization.QuantStub(), resnet18_static_model, torch.quantization.DeQuantStub())
backend = "x86"
quantized_model.qconfig = quant.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
quantized_model = quant.prepare(quantized_model, inplace=True)

/home/tricus/task1/lib/python3.8/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [6]:
# quantization_config = torch.quantization.get_default_qconfig("x86")
# # quantized_model = QuantizedModel(resnet18_static_model) 

# quantized_model.qconfig = quantization_config

# # Print quantization configurations
# print(quantized_model.qconfig)

In [7]:
quantized_model.eval()
with torch.no_grad():
    for images, _ in testloader:
        # images = images.to(device)
        # resnet18_static_model= resnet18_static_model.to(device)

        
        quantized_model(images)

In [8]:
# quantized_model = quant.convert(quantized_model, inplace=True)

In [9]:
# torch.quantization.prepare(quantized_model, inplace=True)

In [10]:
# def calibrate_model(model, loader, device=torch.device("cpu")):

#     model.to(device)
#     model.eval()
    
#     for inputs, labels in loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         _ = model(inputs)



# # Use training data for calibration.

# calibrate_model(model=quantized_model, loader=testloader, device="cpu")

In [11]:
quantized_model = torch.quantization.convert(quantized_model, inplace=True)
quantized_model.eval()

Sequential(
  (0): Quantize(scale=tensor([0.0157]), zero_point=tensor([64]), dtype=torch.quint8)
  (1): ResNet(
    (conv1): QuantizedConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.16462279856204987, zero_point=65, padding=(3, 3), bias=False)
    (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.11506351083517075, zero_point=81, padding=(1, 1), bias=False)
        (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.04344036057591438, zero_point=71, padding=(1, 1), bias=False)
        (bn2): QuantizedBatchNorm2d(64, eps=1e

In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        # images, labels = images.to(device), labels.to(device)
        # trained_resnet18_model= trained_resnet18_model.to(device)
        outputs = quantized_model(images)
    #     _, predicted = torch.max(outputs.data, 1)
    #     total += labels.size(0)
    #     correct += (predicted == labels).sum().item()

    # print('Accuracy of the network on the 10000 test images: %d %%' % (
    #     100 * correct / total))

NotImplementedError: Could not run 'aten::add.out' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::add.out' is only available for these backends: [CPU, Meta, MkldnnCPU, SparseCPU, SparseCsrCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLS